## Homework 5 - Visit the Wikipedia hyperlinks graph!

## Group members:
**Ivana Nastasic <br/>
Musie Meressa Berhe<br/>
Theodoros Sofianos<br/>**

## Introduction:

In this assignment we analysed reduced version of the Wikipedia Hyperlink graph, released by SNAP group. Data is downloaded from following [location](https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view).
Additional information about articles comes from [wiki-topcats](https://snap.stanford.edu/data/wiki-topcats.html).

In [1]:
# Import libraries
import pandas as pd
import networkx as nx
import queue as Q
import statistics as s
import random
import json
import time
import sys

## RQ1 Build the graph G=(V, E), where V is the set of articles and E the hyperlinks among them, and provide its basic information:

-  If it is direct or not <br/>
-  The number of nodes <br/>
-  The number of edges <br/>
-  The average node degree. Is the graph dense?<br/>

To provide the answers on these questions we used Python library [Networkx](https://networkx.github.io/documentation/stable/).<br/>
<br/>
Graph G=(V,E), consisting of the set V of vertices and the set E of edges is **directed**  if edges are ordered pairs of elements of V (i.e. edges are directed from one vertex to another). <br/>
In that definition our input graph is directed. Hyperlinks which represent edges between articles are directed (always going from the source article towards destination article).


In [2]:
# create an empty directed graph
G = nx.DiGraph()

In [3]:
# Read the file with information about article categories
with open('wiki-topcats-categories.txt', 'r') as f:
    cat=f.read().split('\n')

# Make a dictionary with all the categories
# Key is category name and values are number of articles which belong to that category
cat_all={}
try:
    for c_i in cat:
        c_i=c_i.split(';')
        c = c_i[0].split(':')
        cat_all[c[1]] = list(map(int,c_i[1].split()))
except:
    pass

As requested in the assignment description we kept only categories with more than 3500 articles.

In [4]:
categories={}
for key, item in cat_all.items():
    if len(item)>3500:
        categories[key]=item

In [5]:
# Create data frame to be used as an input for placing edges between nodes in the graph
col=['Source','Destination']
df=pd.read_csv('wiki-topcats-reduced.txt',delimiter='\t', names=col)

**We decided to make a graph only using articles (as vertices) which are connected to at least one other article.** <br/>In the input file for categories there are many single articles which are not connected to anything else. As our goal is to calculate the distance between categories based on shortest paths between articles, these articles are not affecting the result.

In [6]:
# Create graph by iterating through all articles which are appearing as a Source (starting point of hyperlink)
# And then placing the edge towards Destination article
# As a feature from add_edge function in networkx, Destination vertx will be directly created if it doesn't already exist 
for index, row, in df.iterrows():
    G.add_node(row['Source'])
    G.add_edge(row['Source'],row['Destination'])

As each article belongs to one or multiple categories, we are storing that information as an attribute **Category** of the vertex.

In [7]:
# set category attribute for each node to empty list
for x in G.nodes():
    G.node[x]['Category']=[]

In [8]:
# Fill in category attribute for each node from the category input list
for key, item in categories.items():
    for i in item:
        try:
            G.node[i]['Category'].append(key)
        except:
            continue

In the later part of the assignment, each article (vertex) will be weighted based on certain criteria.<br/>
In order to be able to store information about the weight additional attribute **Weight** is added and initiated to 0.

In [9]:
for x in G.nodes():
    G.node[x]['Weight']=0

To calculate number of nodes (vertices) we used function ** number_of_nodes**  from networkx library. <br/> In the reduced graph (counting only vertices which have at least one edge) there are ** 461193 ** vertices.

In [207]:
G.number_of_nodes()

461193

To calculate number of edges we used function **number_of_edges** from networkx library. 
There are 2645247 edges.

In [208]:
G.number_of_edges()

2645247

Because we are working with directed graph, we can talk about two types of average vertice degrees, in and out. <br/>
Function **info** from networkx library provides info on both in and out average vertice degree.<br/>
Average in and out degree are equal and value is **5.7357**.

In [212]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

By definition, **dense graph** is a graph in which the number of edges is close to the maximal number of edges. <br/>
The opposite, a graph with only a few edges, is a **sparse graph**. <br/>
The distinction between sparse and dense graphs is rather vague, and depends on the context.<br/>
To calculate density we used **density** function from networkx library.<br/>
Density value is close to 0, therfore graph is sparse.

In [211]:
nx.density(G)

1.2436602635647606e-05

## RQ2 Obtain a block-ranking, where the blocks are represented by the categories.

The first category of the rank, C0, always corresponds to the input category. <br/>
The order of the remaining categories is given by: ** $$ distance(C_{0}, C_{i}) = median(ShortestPath(C_{0}, C_{i})) $$**

To calculate the shortest distance from the given vertex (node) to any other vertex in the graph we implemented **Breadh First Search** algoritham.<br/>

In [10]:

def BFS(Graph,Node):
    distance={} # Dictionary which will have a vertex as a key and shortest distance from the starting vertex as a value
    visited=[False]*1791489 # Goal is to visit all reachable vertices from the start node. Initially all vertices are not visited.
    visited[Node]=True # Start node is marked as visited
    q = Q.Queue() # Queue structure will hold the info about the vertices which are waiting to be visited 
    q.put(Node)
    
    distance[Node] = 0 # Distance from the vertex to itself is 0
    while not q.empty(): # Process continues while there are vertices in the queue which are waiting to be visited
        u = q.get() # Get the next element from the queue
        for i in list(Graph.neighbors(u)): # Check all neighboring vertices
            if not visited[i]: # If neighbour vertex hasn't been visited yet, put it into the queue
                q.put(i)
                visited[i]=True # Mark the vertice as visited
                distance[i] = distance[u]+1 # Graph is initially not weighted, distance between neighbours is 1
    return distance

At this step we are calculating the shortest distance between each vertex of C0 category and each vertex of every other category.<br/> For one vertex of the C0 category, distances to all categories are calculated in one pass through the graph, thanks to the vertex attribute Category. <br/> We tried to avoid passing through the graph too many times because that is time consuming.<br/>
As a starting category C0, we chose **English_television_actors**.

In [11]:
Categories = {} # dictionary which will contain for each category (as a key) list of shortest distances from each vertex of C0 
start = time.time() 
c0=categories['English_television_actors'] # starting category
for x in c0:
    try:
        d = BFS(G,x) # for each vertex from starting category calculate distance to every other accessible vertex
        for j in categories.keys(): # iterate through all the categories
            if(j == 'English_television_actors'):
                Categories.setdefault(category,[]).append(0) # set values to 0 for category C0
            else:
                for k in categories[j]: # for other categories check if vertex appear in the result of shortest distance calculation
                    if(k in d):
                        for category in G.node[k]['Category']: 
                                Categories.setdefault(category,[]).append(d[k]) # if yes, add the distance to the corresponding category
    except:
        pass
end = time.time()
print(end-start) # showing the time needed to calculate all the shortest distances from C0 to all other categories

8278.393723011017


Computation took 138 minutes.

Result is saved into the json file in order to be able to reuse it without repeating time consuming calculation. 

In [ ]:
# save the result into json file
with open('categories_rank', 'w') as fp:
    json.dump(Categories, fp)

While examining the result we noticed that category Living_people is the biggest one, in terms of number of vertices (the most of articles are belonging there). List which contains all the shortest distances between vertices from C0 and that category is huge. <br/>
Calculating the median on the full result was giving a memory error. <br/>
To avoid that problem we calculated median for all the rest of categories skipping Living_people.

In [106]:
# calculating median for each category to be able to rank them
cat_ranking={}
for x in list(Categories.keys()):
    if x!='Living_people':
        cat_ranking.update({x : s.median(Categories[x])})

Then, to calculate score for the category Living_people we used a sample of 1 million shortest distances and calculated median on the sample.

In [128]:
living_people_sample=random.sample(Categories['Living_people'],1000000)
cat_ranking.update({'Living_people' : s.median(living_people_sample)}) # add the value for 'Living_people' into the result

Result is saved into the json file in order to be able to reuse it without recalculation. 

In [128]:
with open('categories_all_median', 'w') as fp:
    json.dump(cat_ranking, fp)

Finally, categories are ranked by the distance from the initial category C0, English_television_actors and result is displayed below.

In [ ]:
sorted_by_value = sorted(cat_ranking.items(), key=lambda kv: kv[1])

In [131]:
sorted_by_value

[('English_television_actors', 4.0),
 ('Article_Feedback_Pilot', 5.0),
 ('People_from_New_York_City', 5),
 ('American_Jews', 5),
 ('American_television_actors', 5.0),
 ('American_film_actors', 5.0),
 ('Harvard_University_alumni', 5),
 ('American_military_personnel_of_World_War_II', 5.0),
 ('British_films', 5.0),
 ('English-language_films', 5.0),
 ('American_films', 5),
 ('Black-and-white_films', 5),
 ('English-language_albums', 5.0),
 ('English_footballers', 6),
 ('The_Football_League_players', 6),
 ('Association_football_forwards', 6.0),
 ('Association_football_goalkeepers', 6.0),
 ('Year_of_death_missing', 6.0),
 ('Association_football_midfielders', 6.0),
 ('Association_football_defenders', 6.0),
 ('English_cricketers', 6.0),
 ('Year_of_birth_missing', 6.0),
 ('Place_of_birth_missing_(living_people)', 6),
 ('Year_of_birth_missing_(living_people)', 6),
 ('Year_of_birth_unknown', 6),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 6.0),
 ('Fellows_of_the_Royal_

## Finally, when obtained the category ranking, we proceeded with sorting the nodes in each category as discribed in the task.<br/>

In the first step, we created subgraph per each category and stored it in the list.

In [145]:
subG=[]
for x in sorted_by_value: 
    subG.append(G.subgraph(categories[x[0]]))

In [ ]:
sorted_nodes=[] # will contain the final result, sorted vertices from all the categories respecting category ranking
visited_nodes=set() # will be used to insure that category of the article corresponds, among the categories it belongs to, to the closest to the input category

Then, we calculated the weight of each vertex in the subgraph of C0 category. <br/>
**Weight is equal to the number of in edges from the vertices which belong to the same category.**

In [174]:
for x in subG[0].nodes(): # iterate through vertices of subgraph for initial category C0
    G.node[x]['Weight'] = subG[0].in_degree(x) # using function in_degree calculate number of incoming edges
# sort the vertices based on the weight
sorted_temp=sorted((nx.get_node_attributes(subG[0],'Weight')).items(), key=lambda kv: kv[1], reverse=True) 
# add sorted values into the result list
for i in sorted_temp:
    sorted_nodes.append(i[0])
# keep all the vertices in the visited_nodes set to insure that they are not used in further categories
visited_nodes.update(sorted_nodes)

Finally, we iterated through the rest of subgraphs (in respect to the category ranking), and calculated weight of each vertex taking into account its' value within the category plus the value of all incoming edges from the previous category.

In [180]:
for i in range(1,len(subG)):
    for x in subG[i].nodes():
        if x not in visited_nodes:  # if we didn't already calculated the weight in the previous category   
            for y in G.in_edges(x): # we look into all incoming edges
                if(y[0] in subG[i].nodes()): # if edge is coming from the same category (same subgraph)
                    G.node[x]['Weight'] +=1 # then weight needs to be increased by 1
                else:
                    G.node[x]['Weight']+=G.node[y[0]]['Weight']  # otherwise weight is increased by the value of the weight of the incoming edge
    sorted_temp=sorted((nx.get_node_attributes(subG[i],'Weight')).items(), key=lambda kv: kv[1], reverse=True) # for the category sort the result
    for j in sorted_temp:
        if j[0] not in visited_nodes: # add sorted values into the result list
            sorted_nodes.append(j[0])
    visited_nodes.update(sorted_nodes) # add all the vertices into the visited_nodes list to insure that they are not used in further categories

Printing the final ranking of the vertices.

In [214]:
sorted_nodes

[1062190,
 1057903,
 1060399,
 1039555,
 1061845,
 1062508,
 1060306,
 1036347,
 1034543,
 1061673,
 1038236,
 1034464,
 1061671,
 1061672,
 1036399,
 1062159,
 1038261,
 1039581,
 1036052,
 1061835,
 1038249,
 1061670,
 1062188,
 1790362,
 1061702,
 1038086,
 1038257,
 1037365,
 1034467,
 1060307,
 1060731,
 1036239,
 1044840,
 1062748,
 1039606,
 1044075,
 1036280,
 1061698,
 1061963,
 1449184,
 1786892,
 1034045,
 1060398,
 1044074,
 1036243,
 1790271,
 1790325,
 1790342,
 1061598,
 1061759,
 1044792,
 1790402,
 1790413,
 1061939,
 1036244,
 1790339,
 1178494,
 1040647,
 1034044,
 1034462,
 1060735,
 1036190,
 1036658,
 1061958,
 1062058,
 1062161,
 1040819,
 1035797,
 1036234,
 1044658,
 1036480,
 1061381,
 1061881,
 1061973,
 1163669,
 1034463,
 1059184,
 1036225,
 1790466,
 1061449,
 1062359,
 1037835,
 1040049,
 1042262,
 1036226,
 1036659,
 1062746,
 1038258,
 1057901,
 1034077,
 1042589,
 1034492,
 1059417,
 1166395,
 1060397,
 1060402,
 1035913,
 1036382,
 1061561,
 1062032,
